In [47]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [9]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [10]:
link = pd.read_csv('./ml-latest-small/links.csv')

In [11]:
rating = pd.read_csv('./ml-latest-small/ratings.csv')

In [12]:
tags = pd.read_csv('./ml-latest-small/tags.csv')

In [13]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [14]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [15]:
movies.title.value_counts()

Saturn 3 (1980)                           2
Eros (2004)                               2
Emma (1996)                               2
Confessions of a Dangerous Mind (2002)    2
War of the Worlds (2005)                  2
                                         ..
Good Earth, The (1937)                    1
Ghost Story (1981)                        1
A Street Cat Named Bob (2016)             1
Why Him? (2016)                           1
Under Suspicion (2000)                    1
Name: title, Length: 9737, dtype: int64

In [16]:
#Check to see if there are any duplicate titles
movies.title.duplicated().sum()

5

In [17]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [18]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [19]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


In [20]:
#Check length of DataFrame

len(movies)

9737

In [21]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

In [22]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [23]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [24]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [25]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [26]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [27]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [28]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

In [29]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [30]:
rating.duplicated().sum()

0

In [31]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [32]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [33]:
movie_rating = movies.join(rating, on='movieId', lsuffix='movie_', rsuffix='rating_', how='outer')

In [34]:
movie_rating

,movieId,movieIdmovie_,genres,Title,year_released,userId,movieIdrating_,rating,timestamp
0.0,1,1.0,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,3.0,4.0,2000-07-30 18:20:47
1.0,2,2.0,Adventure|Children|Fantasy,Jumanji,1995,1.0,6.0,4.0,2000-07-30 18:37:04
2.0,3,3.0,Comedy|Romance,Grumpier Old Men,1995,1.0,47.0,5.0,2000-07-30 19:03:35
3.0,4,4.0,Comedy|Drama|Romance,Waiting to Exhale,1995,1.0,50.0,5.0,2000-07-30 18:48:51
4.0,5,5.0,Comedy,Father of the Bride Part II,1995,1.0,70.0,3.0,2000-07-30 18:40:00
...,...,...,...,...,...,...,...,...,...
NaN,100831,NaN,NaN,NaN,NaN,610.0,166534.0,4.0,2017-05-03 21:53:22
NaN,100832,NaN,NaN,NaN,NaN,610.0,168248.0,5.0,2017-05-03 22:21:31
NaN,100833,NaN,NaN,NaN,NaN,610.0,168250.0,5.0,2017-05-08 19:50:47
NaN,100834,NaN,NaN,NaN,NaN,610.0,168252.0,5.0,2017-05-03 21:19:12


In [35]:
movie_rating.dropna(inplace=True)

In [36]:
movie_rating['userId'].nunique()

397

In [37]:
stats = movie_rating[['rating', 'timestamp']].describe()
stats

,rating
count,8110.000000
mean,3.461467
std,1.094116
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


## Split

In [39]:
train, test = train_test_split(movie_rating)

In [40]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

## Tuning

In [45]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.3s finished


In [46]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 1}, 'mae': {'k': 10, 'min_k': 1}}

In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [49]:
#instantiate KNN model 
model1=KNNBasic()

In [50]:
#instaniatite and perform cross val with training data
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0827  1.0995  1.0878  1.0927  1.0872  1.0900  0.0057  
MAE (testset)     0.8646  0.9047  0.8960  0.8759  0.8763  0.8835  0.0147  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.01    0.00    0.01    0.00    0.01    0.01    0.00    


{'test_rmse': array([1.08268362, 1.09950105, 1.08782774, 1.09270805, 1.0872098 ]),
 'test_mae': array([0.86457331, 0.9047018 , 0.89605   , 0.87585161, 0.87629439]),
 'fit_time': (0.0021228790283203125,
  0.0028192996978759766,
  0.0029571056365966797,
  0.0025992393493652344,
  0.0027518272399902344),
 'test_time': (0.0053539276123046875,
  0.004745006561279297,
  0.0052220821380615234,
  0.004626750946044922,
  0.00534820556640625)}

In [51]:
#use model to predict
model1.predict(uid=10, iid  = 20)
#this model just predicted what user 10 would predict movie 20

Prediction(uid=10, iid=20, r_ui=None, est=3.455815865187012, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

looking at the output above we can determine that our model predicted that use 10 would rate movie 20 a 3.5 give or take our RSME which is 1.09.